In [1]:
import torch
from sklearn import linear_model
# torch.manual_seed(42)
# x = torch.Tensor([1,0,3,0,8,0,0,0,0,0])
# # x = torch.Tensor([0,3,1,2])
# A = torch.randn(3,10)
# A = A/torch.norm(A, dim=0, keepdim=True)
# y = A @ x
x2 = torch.Tensor([0,3,2,1])
x = x2
# X = torch.Tensor([[0,3],[3,2],[1,0],[2,1]])
A = torch.Tensor([[-0.8,0.3,1,0.4],[-0.2,0.4,-0.3,-0.4],[0.2,1,-0.1,0.8]])
A = A/torch.norm(A, dim=0, keepdim=True)
y = A @ x

# y = Dx
def orthogonal_matching_pursuit(compressed_vector, compression_matrix, sparsity,err_treshold=1e-6):
    # initialize
    # compression_matrix_norm = torch.norm(compression_matrix, dim=0, keepdim=True)
    unit_compression_matrix = compression_matrix 
    subsapce_index_list = []
    residual = compressed_vector
    k = 1
    sparsity_vecotr = torch.zeros(compression_matrix.shape[1])
    err = torch.sum(residual**2)
    while k <= sparsity and err > err_treshold:
        residual_projection_matrix = torch.abs(unit_compression_matrix.T @ residual)
        _, matching_basis_index = torch.max(residual_projection_matrix, dim=0)
        subsapce_index_list.append(matching_basis_index.item())
        subspace_matrix = compression_matrix[:, subsapce_index_list]
        subspace_matrix_pinv = torch.pinverse(subspace_matrix)
        subspace_coordinates = subspace_matrix_pinv @ compressed_vector
        subspace_representation = subspace_matrix @ subspace_coordinates
        residual = compressed_vector - subspace_representation
        err = torch.sum(residual**2)
        k += 1
    sparsity_vecotr[subsapce_index_list] = subspace_coordinates
    return sparsity_vecotr

orthogonal_matching_pursuit(y,A, sparsity=4,err_treshold=1e-6), linear_model.orthogonal_mp(A.numpy(),y.numpy(), n_nonzero_coefs=4, tol=1e-6)
# len(torch.nonzero(torch.zeros(7,7)))

(tensor([-1.7058,  1.9212,  0.0000,  2.4407]),
 array([-1.70580685,  1.92115259,  0.        ,  2.44069219]))

In [169]:
import torch
x1 = torch.Tensor([0,3,1,2])
x2 = torch.Tensor([0,3,2,1])

X = torch.stack([x1,x2], dim=1)
# X = torch.Tensor([[0,3],[3,2],[1,0],[2,1]])
A = torch.Tensor([[-0.8,0.3,1,0.4],[-0.2,0.4,-0.3,-0.4],[0.2,1,-0.1,0.8]])
A = A/torch.norm(A, dim=0, keepdim=True)
Y = A @ X

_, origin_dim = A.shape
_, sample_num = Y.shape
sparisty_matrix = torch.zeros(origin_dim, sample_num)

for sample_index in range(sample_num):
    y = Y[:,sample_index]
    sparisty_matrix[:,sample_index] = orthogonal_matching_pursuit(y, A, sparsity=4, err_treshold=1e-6)
sparisty_matrix, linear_model.orthogonal_mp(A.numpy(),Y.numpy(), n_nonzero_coefs=4, tol=1e-6)

(tensor([[-0.8529, -1.7058],
         [ 2.4606,  1.9212],
         [ 0.0000,  0.0000],
         [ 2.7203,  2.4407]]),
 array([[-0.85290349, -1.70580673],
        [ 2.46057558,  1.92115247],
        [ 0.        ,  0.        ],
        [ 2.72034669,  2.44069266]]))

In [56]:
import torch
from sklearn import linear_model

def orthogonal_matching_pursuit(
    compressed_vector, compression_matrix, sparsity, err_treshold=None
):
    '''
    Args:
        compressed_vector: (compression_dim, )
        compression_matrix: (compression_dim, sample_dim)
    Returns:
        sparsity_vecotr: (sample_dim, )
    '''
    # initialize
    # compression_matrix_norm = torch.norm(compression_matrix, dim=0, keepdim=True)
    unit_compression_matrix = compression_matrix
    subsapce_index_list = []
    residual = compressed_vector
    k = 1
    sparsity_vecotr = torch.zeros(
        compression_matrix.shape[1], device=compressed_vector.device
    )
    while k <= sparsity:
        residual_projection_matrix = torch.abs(unit_compression_matrix.T @ residual)
        _, matching_basis_index = torch.max(residual_projection_matrix, dim=0)
        subsapce_index_list.append(matching_basis_index.item())
        subspace_matrix = compression_matrix[:, subsapce_index_list]
        subspace_matrix_pinv = torch.pinverse(subspace_matrix)
        subspace_coordinates = subspace_matrix_pinv @ compressed_vector
        subspace_representation = subspace_matrix @ subspace_coordinates
        residual = compressed_vector - subspace_representation
        err = torch.sum(residual**2)
        if err_treshold is not None and err <= err_treshold:
            break
        k += 1
    sparsity_vecotr[subsapce_index_list] = subspace_coordinates
    return sparsity_vecotr


def orthogonal_matching_pursuit_matrix(
    compressed_matrix, compression_matrix, sparsity, err_treshold=None
):
    '''
    Args:
        compressed_matrix: (compression_dim, sample_num)
        compression_matrix: (compression_dim, sample_dim)
    Returns:
        sparsity_matrix: (sample_dim, sample_num)
    '''
    _, sample_num = compressed_matrix.shape
    _, sample_dim = compression_matrix.shape
    sparsity_matrix = torch.zeros(
        sample_dim, sample_num, device=compressed_matrix.device
    )
    for sample_idx in range(sample_num):
        compressed_vector = compressed_matrix[:, sample_idx]
        sparsity_vecotr = orthogonal_matching_pursuit(
            compressed_vector,
            compression_matrix,
            sparsity=sparsity,
            err_treshold=err_treshold,
        )
        sparsity_matrix[:, sample_idx] = sparsity_vecotr

    return sparsity_matrix


x1 = torch.Tensor([0,3,1,2])
x2 = torch.Tensor([0,3,2,1])

X = torch.rand(256,4096).to('cpu')
# X = torch.Tensor([[0,3],[3,2],[1,0],[2,1]])
A = torch.rand(64,256).to('cpu')
A = A/torch.norm(A, dim=0, keepdim=True)
Y = A @ X



In [57]:
S1 = orthogonal_matching_pursuit_matrix(Y,A,sparsity=25)

In [50]:
S2 = linear_model.orthogonal_mp(A.cpu().numpy(),Y.cpu().numpy(), n_nonzero_coefs=25)

In [37]:
S1.shape, S2.shape

(torch.Size([256, 1024]), (256, 1024))

In [52]:
S1[:20,0], S2[:,0]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 6.8686,
         2.1188, 4.4725]),
 array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  6.86864614,  2.11873317,  4.47249603,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  3.46769309,  0.        ,  0.        ,  2.44842958,
         5.37361908,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  7.39130545,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        

In [ ]:
import sys
sys.path.append('..')
from skimage import io,data
from src.model.spstfm.ksvd import KSVD
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

img=data.astronaut()
rng_seed = 42
torch.manual_seed(rng_seed)
torch.cuda.manual_seed(rng_seed)
# io.imshow(img)

# img_train = img[:256, :, :]
# io.imshow(img_train)
device = 'cuda'
n_components = 256
ksvd = KSVD(n_components=n_components,max_iter=100,tol=1e-6,sparsity=10).to(device)
img = img.astype('float') / 255.0
img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).float().to(device)
img_norm = (img - torch.mean(img,dim=(-2,-1),keepdim=True)) / (torch.std(img,dim=(-2,-1),keepdim=True) + 1e-10)
img_patch = torch.nn.Unfold(kernel_size=8, stride=8)(img_norm)
img_patch = img_patch.reshape(3, 64, -1)
dictionary_matrix = torch.zeros(3, 64, n_components,device=device)
sparsity_matrix = torch.zeros(3, n_components, img_patch.shape[-1],device=device)
for i in range(3):
    img_patch_per_channel = img_patch[i]
    dictionary_matrix_per_channel, sparsity_matrix_per_channel = ksvd.fit(img_patch_per_channel)
    dictionary_matrix[i] = dictionary_matrix_per_channel
    sparsity_matrix[i] = sparsity_matrix_per_channel

In [60]:
import numpy as np
import torch

a = np.random.rand(1,1,3,4)
torch.nn.Unfold(kernel_size=2, stride=2)(torch.from_numpy(a)).shape

AttributeError: 'numpy.ndarray' object has no attribute 'unfold'

In [ ]:
import torch
from sklearn import linear_model

